## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import keras
import os
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
BATCH_SIZE = batch_size
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環
EPOCHS = epochs

In [4]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
# model.summary()

In [5]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
train_set = data_generator.flow(
    x_train, y_train,
    batch_size = BATCH_SIZE,
    shuffle=True
)

val_set = data_generator.flow(
    x_test, y_test,
    shuffle = False,
    batch_size = 256
)

num_train = train_set.n
num_val = val_set.n

In [6]:
model.fit_generator(
    train_set,
    steps_per_epoch  = num_train // BATCH_SIZE,
    epochs = EPOCHS,
    verbose = 1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
781/781 [==============================] - 95s 121ms/step - loss: 1.9916 - acc: 0.4515
Epoch 2/30
781/781 [==============================] - 87s 112ms/step - loss: 1.5588 - acc: 0.5718
Epoch 3/30
781/781 [==============================] - 86s 111ms/step - loss: 1.3740 - acc: 0.6230
Epoch 4/30
781/781 [==============================] - 86s 110ms/step - loss: 1.2496 - acc: 0.6596
Epoch 5/30
781/781 [==============================] - 86s 111ms/step - loss: 1.1659 - acc: 0.6876
Epoch 6/30
781/781 [==============================] - 87s 111ms/step - loss: 1.0897 - acc: 0.7108
Epoch 7/30
781/781 [==============================] - 87s 111ms/step - loss: 1.0338 - acc: 0.7325
Epoch 8/30
781/781 [==============================] - 87s 111ms/step - loss: 0.9918 - acc: 0.7457
Epoch 9/30
781/781 [==============================] - 87s 111ms/step - loss: 0.9551 - acc: 0.7574
Epoch 10/30
781/781 [=

In [7]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8547270872116088
Test accuracy: 0.8097


In [5]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 105s 2ms/step - loss: 1.8292 - acc: 0.5151 - val_loss: 1.6509 - val_acc: 0.5626
Epoch 2/30
50000/50000 [==============================] - 92s 2ms/step - loss: 1.3276 - acc: 0.6581 - val_loss: 1.6675 - val_acc: 0.5693
Epoch 3/30
50000/50000 [==============================] - 92s 2ms/step - loss: 1.1117 - acc: 0.7252 - val_loss: 1.3627 - val_acc: 0.6270
Epoch 4/30
50000/50000 [==============================] - 92s 2ms/step - loss: 0.9837 - acc: 0.7645 - val_loss: 1.5374 - val_acc: 0.6215
Epoch 5/30
50000/50000 [==============================] - 92s 2ms/step - loss: 0.8890 - acc: 0.7953 - val_loss: 1.6725 - val_acc: 0.5746
Epoch 6/30
50000/50000 [==============================] - 92s 2ms/step - loss: 0.8185 - acc: 0.8185 - val_loss: 1.3908 - val_acc: 0.6609
Epoch 7/30
50000/50000 [=======

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.35731405916214
Test accuracy: 0.7595
